## TASK 3: Quantum Convolutional Neural Network (QCNN)

Dataset: electron-photon.npz

Task is to classify the images as 0 if photon and 1 if electron and to show the training loss decreases over time and any other model tuning.

- This notebook consists of working implementation of QCNN for the given dataset and at the end hyper parameter tuning is shown
- Used code snippets from tutorials of tensorflow Quantum
- Skip the following block, if tensorflow and tensorflow quantum are installed already.


In [ ]:
!pip install tensorflow==2.3.1
!pip install tensorflow_quantum

In [ ]:

import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

### Loading Data

Get training data, labels and test data and corresponding labels

The .npz file consists of zero-dimensional array, accessing which requires expanding its dimensions along axis=0

In [ ]:
data = np.load("/content/electron-photon.npz", allow_pickle=True)
total_data = dict(zip(("x_train","y_train","x_test", "y_test"), (data[k] for k in data)))
total_data = np.expand_dims(total_data,axis=0)

x_test = total_data[0]['x_test']
y_test = total_data[0]['y_test']
x_train = total_data[0]['x_train']
y_train = total_data[0]['y_train']
 

Encoding of classical data into qubits

In [ ]:
def generate_data(qubits, training_data, train_labels, test_data, test_labels):
    """Generate training and testing data."""
    train_excitations = []
    test_excitations = []
    labels = []
    for data in training_data:
        bit = np.random.randint(0,len(qubits))
        rng = np.average(data)*(np.pi)
        train_excitations.append(cirq.Circuit(cirq.rx(rng)(qubits[bit])))
    for data in test_data:
        bit = np.random.randint(0,len(qubits))
        rng = np.average(data)*(np.pi)
        test_excitations.append(cirq.Circuit(cirq.rx(rng)(qubits[bit])))

    return tfq.convert_to_tensor(train_excitations), np.array(train_labels), \
        tfq.convert_to_tensor(test_excitations), np.array(test_labels)


 Preparing Cluster state

In [ ]:
def cluster_state_circuit(bits):
    """Return a cluster state on the qubits in `bits`."""
    circuit = cirq.Circuit()
    circuit.append(cirq.H.on_each(bits))
    for this_bit, next_bit in zip(bits, bits[1:] + [bits[0]]):
        circuit.append(cirq.CZ(this_bit, next_bit))
    return circuit

Helper functions for creating single and two qubit unitaries and pooling operation.

In [ ]:
def one_qubit_unitary(bit, symbols):
    """Make a Cirq circuit enacting a rotation of the bloch sphere about the X,
    Y and Z axis, that depends on the values in `symbols`.
    """
    return cirq.Circuit(
        cirq.X(bit)**symbols[0],
        cirq.Y(bit)**symbols[1],
        cirq.Z(bit)**symbols[2])


def two_qubit_unitary(bits, symbols):
    """Make a Cirq circuit that creates an arbitrary two qubit unitary."""
    circuit = cirq.Circuit()
    circuit += one_qubit_unitary(bits[0], symbols[0:3])
    circuit += one_qubit_unitary(bits[1], symbols[3:6])
    circuit += [cirq.ZZ(*bits)**symbols[6]]
    circuit += [cirq.YY(*bits)**symbols[7]]
    circuit += [cirq.XX(*bits)**symbols[8]]
    circuit += one_qubit_unitary(bits[0], symbols[9:12])
    circuit += one_qubit_unitary(bits[1], symbols[12:])
    return circuit


def two_qubit_pool(source_qubit, sink_qubit, symbols):
    """Make a Cirq circuit to do a parameterized 'pooling' operation, which
    attempts to reduce entanglement down from two qubits to just one."""
    pool_circuit = cirq.Circuit()
    sink_basis_selector = one_qubit_unitary(sink_qubit, symbols[0:3])
    source_basis_selector = one_qubit_unitary(source_qubit, symbols[3:6])
    pool_circuit.append(sink_basis_selector)
    pool_circuit.append(source_basis_selector)
    pool_circuit.append(cirq.CNOT(control=source_qubit, target=sink_qubit))
    pool_circuit.append(sink_basis_selector**-1)
    return pool_circuit


In [ ]:
def quantum_conv_circuit(bits, symbols):
    """Quantum Convolution Layer following the above diagram.
    Return a Cirq circuit with the cascade of `two_qubit_unitary` applied
    to all pairs of qubits in `bits` as in the diagram above.
    """
    circuit = cirq.Circuit()
    for first, second in zip(bits[0::2], bits[1::2]):
        circuit += two_qubit_unitary([first, second], symbols)
    for first, second in zip(bits[1::2], bits[2::2] + [bits[0]]):
        circuit += two_qubit_unitary([first, second], symbols)
    return circuit

In [ ]:
def quantum_pool_circuit(source_bits, sink_bits, symbols):
    """A layer that specifies a quantum pooling operation.
    A Quantum pool tries to learn to pool the relevant information from two
    qubits onto 1.
    """
    circuit = cirq.Circuit()
    for source, sink in zip(source_bits, sink_bits):
        circuit += two_qubit_pool(source, sink, symbols)
    return circuit

The core architecture of QCNN

In [ ]:
def create_model_circuit(qubits):
    """Create sequence of alternating convolution and pooling operators 
    which gradually shrink over time."""
    model_circuit = cirq.Circuit()
    symbols = sympy.symbols('qconv0:63')

    model_circuit += quantum_conv_circuit(qubits, symbols[0:15])
    model_circuit += quantum_pool_circuit(qubits[:4], qubits[4:],
                                          symbols[15:21])
    model_circuit += quantum_conv_circuit(qubits, symbols[0:15])
    model_circuit += quantum_conv_circuit(qubits[4:], symbols[21:36])
    model_circuit += quantum_pool_circuit(qubits[4:6], qubits[6:],
                                          symbols[36:42])
    model_circuit += quantum_conv_circuit(qubits, symbols[0:15])
    model_circuit += quantum_conv_circuit(qubits[6:], symbols[42:57])
    
    model_circuit += quantum_pool_circuit([qubits[6]], [qubits[7]],
                                          symbols[57:63])
    return model_circuit




Adding Controller to change Parameterized Quantum Circuit based on input taken through commands_input.

In [ ]:
controller = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation=tf.nn.softmax),
])


commands_input = tf.keras.Input(shape=(32,32),
                                dtype=tf.dtypes.float32,
                                name='commands_input')


Following code plots the abstract view of the architecture

In [ ]:
cluster_state_bits = cirq.GridQubit.rect(1, 8)
readout_operators = cirq.Z(cluster_state_bits[-1])

excitation_input = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
cluster_state = tfq.layers.AddCircuit()(
    excitation_input, prepend=cluster_state_circuit(cluster_state_bits))

quantum_model = tfq.layers.PQC(create_model_circuit(cluster_state_bits),
                               readout_operators)(cluster_state)

qcnn_model = tf.keras.Model(inputs=[excitation_input, commands_input], outputs=[quantum_model])

tf.keras.utils.plot_model(qcnn_model,
                          show_shapes=True,
                          show_layer_names=False,
                          dpi=70)

Adding tensorboard for monitoring

In [ ]:
%load_ext tensorboard
import datetime

log_dir = "logs_tb/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



Created custom accuracy metric and evaluating the model

In [ ]:
# Generate some training data.
train_excitations, train_labels, test_excitations, test_labels = generate_data(
    cluster_state_bits, x_train,y_train,x_test,y_test)

# Custom accuracy metric.
@tf.function
def custom_accuracy(y_true, y_pred):  
    y_true = tf.squeeze(y_true)
    y_pred = tf.map_fn(lambda x: 1.0 if x >= 0.4 else 0.0, y_pred)
    return tf.keras.backend.mean(tf.keras.backend.equal(y_true, y_pred))


qcnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
                   loss=tf.losses.mse,
                   metrics=[custom_accuracy])

split_ind = int(0.8*len(x_train))
history = qcnn_model.fit(x=[train_excitations[:split_ind], x_train[:split_ind]],
                         y=y_train[:split_ind],
                         batch_size=16,
                         epochs=25,
                         verbose=1,
                         validation_data=([train_excitations[split_ind:], x_train[split_ind:]], y_train[split_ind:]),
                         callbacks=tb_callback
                         )

res = qcnn_model.evaluate(x=[test_excitations,x_test], y=y_test)

print(f"loss: {res[0]} accuracy= {res[1]}")



Plotting training and validation loss vs Epochs

In [ ]:
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Training a Quantum CNN on electron photon data')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Utilizing hyper-parameter tuning to run experiments and tune the model. You can observe the corresponding results in log directory located at logs_tb/hparam_tuning 

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([10, 20]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs_tb/hparam_tuning').as_default():
      hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
      )


In [ ]:
def train_test_model(hparams, log_dir):
    controller = tf.keras.Sequential([
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
      tf.keras.layers.Dense(8, activation=tf.nn.softmax),
  ])

    commands_input = tf.keras.Input(shape=(32,32),
                                  dtype=tf.dtypes.float32,
                                  name='commands_input')
    cluster_state_bits = cirq.GridQubit.rect(1, 8)
    readout_operators = cirq.Z(cluster_state_bits[-1])

    excitation_input = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
    cluster_state = tfq.layers.AddCircuit()(
      excitation_input, prepend=cluster_state_circuit(cluster_state_bits))

    quantum_model = tfq.layers.PQC(create_model_circuit(cluster_state_bits),
                                readout_operators)(cluster_state)



    qcnn_model = tf.keras.Model(inputs=[excitation_input, commands_input], outputs=[quantum_model])
    qcnn_model.compile(optimizer=hparams[HP_OPTIMIZER],
                    loss=tf.losses.mse,
                    metrics=[custom_accuracy])

    split_ind = int(0.8*len(x_train))
    history = qcnn_model.fit(x=[train_excitations[:split_ind], x_train[:split_ind]],
                          y=y_train[:split_ind],
                          batch_size=16,
                          epochs=25,
                          verbose=1,
                          validation_data=([train_excitations[split_ind:], x_train[split_ind:]], y_train[split_ind:]),
                           callbacks=[tb_callback,
                                      hp.KerasCallback(log_dir, hparams),] # log hparams

                          )

    [loss, accuracy] = qcnn_model.evaluate(x=[test_excitations,x_test], y=y_test)
    return accuracy

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
              HP_NUM_UNITS: num_units,
              HP_DROPOUT: dropout_rate,
              HP_OPTIMIZER: optimizer,
          }

        print(f"{hparams}")
        accuracy = train_test_model(hparams,'logs_tb/hparam_tuning/run'+str(session_num))
        session_num += 1
